# Mystery box - 1 - GPS data check
### Dennis Bakhuis - 8th September 2022
### https://linkedin.com/in/dennisbakhuis/

Checking if the GPS data makes sense and fixing the time format.

In [ ]:
import pytz

import gpxpy
import pandas as pd
import folium

## First step: importing the data using `gpxpy`

In [ ]:
gps_file = '../data/raw/Session22/20221019-171139 - Rondje 22.gpx'

with open(gps_file, 'r') as f:
    gpx = gpxpy.parse(f)

## Next: converting it into a DataFrame

In [ ]:
points = []
for segment in gpx.tracks[0].segments:
    for p in segment.points:
        points.append({
            'time': p.time,
            'latitude': p.latitude,
            'longitude': p.longitude,
            'elevation': p.elevation,
        })

df = pd.DataFrame.from_records(points)

### Check how the GPS data looks

In [ ]:
map_enschede = folium.Map(
    location=[52.219433827394745, 6.896132071889914], 
    zoom_start=14, 
    tiles="Stamen Watercolor",
)

points = df[['latitude', 'longitude']].values.tolist()

folium.PolyLine(
    points, 
    color="red", 
    weight=5, 
    opacity=1
).add_to(map_enschede)

map_enschede

### Fix time

In [ ]:
df = (
    df
    .set_index('time')
    .tz_convert('Europe/Amsterdam')
    .tz_localize(None)
    .reset_index()
)

In [ ]:
df.to_parquet('../data/mystery_box/gps_data.parquet', index=False)